In [ ]:
import pandas as pd
df1=pd.read_csv("/content/train.csv")
df2=pd.read_csv("/content/test.csv")

# Combine using concat()
df_train = pd.concat([df1, df2])

In [ ]:
df_train.head()

,category,sub_category,crimeaditionalinfo
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...


In [ ]:
print("Dataset Shape:", df_train.shape)
print("\nColumns in the dataset:", df_train.columns.tolist())

Dataset Shape: (124915, 3)

Columns in the dataset: ['category', 'sub_category', 'crimeaditionalinfo']


### **Empty or whitespace values, missing values, duplicates, charecters length < 150**

In [ ]:
df_train_cleaned = df_train.copy()
df_train_cleaned.dropna(subset=['crimeaditionalinfo'] ,inplace=True)

In [ ]:
# 1. Empty or whitespace values
def strip_warn(row):
    try:
        x = row.strip()
        return x
    except:
        print(row)
        return ""


print(df_train_cleaned[df_train_cleaned['crimeaditionalinfo'].str.strip().str.len() == 0].shape)
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(strip_warn)
df_train_cleaned = df_train_cleaned[df_train_cleaned['crimeaditionalinfo'].str.strip().str.len() > 0]

(1582, 3)


In [ ]:
# 2. Missing values
print(df_train_cleaned.isnull().sum())

category                 0
sub_category          8758
crimeaditionalinfo       0
dtype: int64


In [ ]:
# df_train_cleaned=df_train_cleaned.dropna(subset=["crimeaditionalinfo"])

In [ ]:
# 3. Duplicates
print(df_train_cleaned.duplicated().sum())

11295


In [ ]:
df_train_cleaned=df_train_cleaned.drop_duplicates()

In [ ]:
print(df_train_cleaned['crimeaditionalinfo'].duplicated().sum())

643


In [ ]:
df_train_cleaned = df_train_cleaned.drop_duplicates(subset=['crimeaditionalinfo'])

In [ ]:
# 4. Character length check
count_less_than_150 = df_train_cleaned[df_train_cleaned['crimeaditionalinfo'].str.len() <= 150].shape[0]
print("Number of rows with less than 150 characters in the crimeaditionalinfo column:", count_less_than_150)

Number of rows with less than 150 characters in the crimeaditionalinfo column: 8237


In [ ]:
# Keep only rows where text length is > 150 characters
df_train_cleaned = df_train_cleaned[df_train_cleaned['crimeaditionalinfo'].str.len() > 150]

# Reset index
df_train_cleaned.reset_index(drop=True, inplace=True)

In [ ]:
df_train_cleaned.shape

(103130, 3)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_train_cleaned.sample(10)

,category,sub_category,crimeaditionalinfo
45585,Online Financial Fraud,Internet Banking Related Fraud,Online Frauds Advertisement Portal Fraud eCommerce platform OLX\r\nvictim posted a ad of product in OLX to sell it fraudster approached him as buyer and sent QR codes to pay amount victim scanned the QR codes and lost money instead of getting money
94992,Online Financial Fraud,UPI Related Frauds,Respected Sir today suddenly I received sms of deduction amount from my PayTm account I did not any activity with phone or with my account Neither I share OTP nor I make any order Without Sharing any details amount of Rs deducted from PayTm
48152,Online Financial Fraud,Fraud CallVishing,sir i had taken loan from express app its due date was on th jan i made payment but it was not updated then forced again todo payment then i did extension by paying now the due date came again on th i made the payment but it was not updated again took my two time i have made all these payment through this app still these people are doing fraud these are all number sir these companykeep changing their upi id every hoursthese people are now abusing sir dirty dirty and threating to send porn videos to relative contacts pls hlp me sir
99167,Cyber Attack/ Dependent Crimes,Tampering with computer source documents,"I never imagined something like this could happen to me, especially since I'm so cautious. But then I started noticing that my online activity felt different, slower even. I can't even describe it well because everything seemed normal, except for the fact that I was constantly getting logged out of my accounts. Looking back, maybe I should have paid more attention to the little details, but now it's too late. My accounts are compromised. The more I tried to fix it, the more problems came up. My bank account was recived, and then I got locked out of everything. I feel so helpless, like there’s nothing I can do to fix this. No matter how many times I change my password, it’s like someone’s always one step ahead. Even my phone isn’t working right anymore. It’s so stange how everything just fell apart after that one email."
36097,Online Financial Fraud,UPI Related Frauds,Message came with phone pe logo and they asked that i have received cash back then i scrach the coupon then they asked for sending money to your account i clicked that icon then phone pe opened asked for UPI and then Money debited from my account
16710,Online Financial Fraud,UPI Related Frauds,Respected one\r\nthis incidence occured times previously from the same person firstvtime they took Rs from my account as per refund money but it didnt refund back and i didnt have any trasaction history about this but they called me to take my information \r\nPlease accept my request and procide futher\r\nhoping for positive response
64139,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,On th Feb night on reaching home from the office I opened my FB account to check some messages from my industry contacts I got a friend request from an anonymous girl named Radhika from Nasik Since she looked familiar I accepted the request and she started chatting on FB Messenger I just wanted to know who she was I exchanged messages and WhatsApp no was shared mutually to contact Immediately I started getting messages on WhatsApp and she asked for a video call I grew suspicious but on a Fair Note I wanted to know about her and accepted to take the call She started the VIDEO Call and became NUDE and in a few seconds I disconnected the call In less than a minute the recorded VIDEO got shared with me and some anonymous male got in touch and asked to pay Rs for not to upload this onto FB Since I did not want to take any chances and got under PRESSURE paid Rs Later again I got another message and audio recording asking to pay another and the messages got exchanged with that person till almost pm Finally I had paid more than to close the issue But to my surprise I started receiving calls msgs now D

### **Reduce noise in data**

In [ ]:
! pip install contractions -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 8.8 MB/s eta 0:00:00


In [ ]:
# Import required libraries
import re
import unicodedata
import contractions


# Fix encoding and handle accented characters
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: unicodedata.normalize('NFKD', x))
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: x.encode('ascii', 'ignore').decode('utf-8'))

# Expand contractions
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: contractions.fix(x))

# Remove noise but keep periods
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: x.lower())
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: re.sub(r'\S+@\S+', '', x))
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: re.sub(r'http\S+|www.\S+', '', x))
# Modified to keep periods while removing other punctuation
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: re.sub(r'[^\w\s\.]', '', x))
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: re.sub(r'\s+', ' ', x))
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: x.strip())

In [ ]:
df_train_cleaned.sample(10)

,category,sub_category,crimeaditionalinfo
96592,Online and Social Media Related Crime,Cheating by Impersonation,elza me mene payment kiya tha tb bola tha muje ki bin ka aaye ga din tak or abhi mere account me withdraw nhi ho rha he roj skim dete he ki etne lgao to payment wapas account aaye he
45305,Cyber Attack/ Dependent Crimes,Denial of Service (DoS)/Distributed Denial of Service (DDOS) attacks,this all happened a few days after i accidentally clicked on a strange link on social media. i did not think much of it until things started going wrong. the next thing i knew people were calling me asking if i had sent them strange messages and i had not touched my phone all day. honestly its such a mess and i do not even know where to start. i have contacted support but they keep giving me the runaround. the more i tried to fix it the more problems came up. my email was notifiction and then i got locked out of everything. this has really left me anxious and frustrated i cannot sleep knowing my information is out there. i have even considered deleting all my accounts but that is not a solution. even my laptop is not working right anymore. its so stange how everything just fell apart after that one email.
82322,Cryptocurrency Crime,Cryptocurrency Fraud,whatsapp number and i have transferred amount from binance account they not refunding my amount kindly check and help me i have transferred dollar make a complain pls help me
57402,Online and Social Media Related Crime,EMail Phishing,an email impersonating my university was received by me which asked to fill a google form and they sought to fill my details like aadhar my admission details etc i being unknown of the fact that it was fake filled the form and submitted it
33342,Online and Social Media Related Crime,FakeImpersonating Profile,dear sir my name is mdsalim so late mdnasimuddin and i am presently live in kolkata narayapur rajarhat pin and my home town jamshedpur east singhbhum jharkand pin yesterday on dated jan at pm i have received calls from my friends frequently somebody used my facebook fake ids with my photo and asked money chat details provide me through whatapps same uploaded in your website during chatting identified two mobile numbers and pytm name of numbers name rohit singh account numbers ifsc code pytm upi id paytm fake facebook fake id profile sunilkumarsunilkumsr and my facebook profile id requesting kindly investigate the matter and take strong action
89824,Any Other Cyber Crime,Other,we sent our children to their aunty house we gave the phone also to attend online class when ready to go home they kept phone in their bag but reached our home we check their bag we not found the phone so we made call evening time the phone was switchoff but night time the phone was ringing next days also the phone was ringing but rd day the phone was switchoff
94504,Online Financial Fraud,UPI Related Frauds,i posted an add on facebook to sell my wardrobe and got a call from indian army asking for price and he agreed to buy for advance payment he sent a link to accept the creditwhich resulted in fraud of k
94291,Sexually Obscene material,NaN,dear sir please help us this message from small loan o cash application i was take loan and day overdue but their recovery agent is still abusing and harrassing me like this on what is app they send message like this to everyone sir i trust you you will take immediate action sir jai hind
66923,Online Financial Fraud,Internet Banking Related Fraud,identity theft fake customer care service fraud any othersvictim saw a add inyou tube about bajaj finance loan victim contacted to bajaj customer care number and connecting later automatically it is disconnecting after some time victim got call from unknown number he introduced him self as a bajaj employee so victim believed his words and victim shares card details and download any desk app later amount was debited from his account
58071,Any Other Cyber Crime,Other,my husband went to pnb atm outlet at

Language detection for stop word removal
(on hold)

In [ ]:
'''!pip install indic-nlp-library langdetect
!pip install polyglot
!pip install pyicu
!pip install pycld2

import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize import indic_normalize
import re
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Normalize Indic text
    if lang in ["hi", "bn", "ta", "te", "mr"]:
        normalizer = indic_normalize.Normalizer()
        text = normalizer.normalize(text)

    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    # Tokenization
    words = text.split() if lang == "en" else indic_tokenize.trivial_tokenize(text)

    # Stopword removal and lemmatization
    if lang == "en":
        words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    elif lang in ["hi", "bn", "ta", "te", "mr"]:
        words = [word for word in words if word not in indic_stopwords]

    return " ".join(words)

'''

'!pip install indic-nlp-library langdetect\n!pip install polyglot\n!pip install pyicu\n!pip install pycld2\n\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom nltk.corpus import stopwords\nfrom nltk.stem import WordNetLemmatizer\nfrom indicnlp.tokenize import indic_tokenize\nfrom indicnlp.normalize import indic_normalize\nimport re\nimport nltk\n\nnltk.download(\'stopwords\')\nnltk.download(\'wordnet\')\nnltk.download(\'punkt\')\n\ndef preprocess_text(text):\n    if not isinstance(text, str):\n        return ""\n\n    # Normalize Indic text\n    if lang in ["hi", "bn", "ta", "te", "mr"]:\n        normalizer = indic_normalize.Normalizer()\n        text = normalizer.normalize(text)\n\n    # Remove special characters and digits\n    text = re.sub(r\'[^\\w\\s]\', \'\', text)\n    text = re.sub(r\'\\d+\', \'\', text)\n\n    # Tokenization\n    words = text.split() if lang == "en" else indic_tokenize.trivial_tokenize(text)\n\n    # Stopword removal and lemmatization\n    if lang ==

## **Stop-word removal, lemmatization, vectorization, tokenization, normalization**

In [ ]:
! pip install nltk indic-nlp-library cleanlab unidecode autocorrect -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 6.9 MB/s eta 0:00:00


In [ ]:
import re
import nltk
import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
indic_stopwords = set([
    "aur", "ki", "hai", "huyi", "ho", "mein", "ye", "ke", "jo", "saath", "ko",
    "bhi", "tatha", "par", "se", "kisi", "un", "apna", "tum", "main", "aap", "inhe",
    "in", "abhi", "ab", "woh", "hum", "unka", "is", "us", "kintu", "athva", "nahin",
    "kar","firto","fir","kese","esse","ka","kabhi","karna"
])
stop_words=stop_words.union(indic_stopwords)

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Normalize Unicode characters
    text = unidecode.unidecode(text)

    # Tokenization
    words = word_tokenize(text)

    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]

    # Removing stopwords
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)

In [ ]:
# Apply preprocessing
df_train_cleaned["crimeaditionalinfo"] = df_train_cleaned.apply(lambda row: preprocess_text(row["crimeaditionalinfo"]), axis=1)


### Word Embeddings

In [ ]:
!pip install tqdm -q
from tqdm.auto import tqdm  # Import tqdm
import pandas as pd

tqdm.pandas()  # Integrate tqdm with pandas

sentences = df_train_cleaned['crimeaditionalinfo']
train_sentences = list(sentences.progress_apply(str.split).values) # Now progress_apply will work

  0%|          | 0/103130 [00:00<?, ?it/s]

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import time

from gensim.models import Word2Vec
from tqdm import tqdm

tqdm.pandas()

In [ ]:
start_time = time.time()

model = Word2Vec(sentences=train_sentences,
                 sg=1,
                 vector_size=100,
                 workers=4)

print(f'Time taken : {(time.time() - start_time) / 60:.2f} mins')

Time taken : 2.31 mins


In [ ]:
len(model.wv.vocab.keys())

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [ ]:
model.wv.vector_size

In [ ]:
model.wv.get_vector('iphone')

## Bakwas

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import cleanlab
from cleanlab.classification import CleanLearning
from sklearn.linear_model import LogisticRegression

In [ ]:
# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_cleaned["crimeaditionalinfo"])

In [ ]:
'''
#Dont run unless you really have to check
tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names_out())

final_df = tf_idf

print("{} rows".format(final_df.shape[0]))
final_df.T.nlargest(5, 0)
'''

'\n#Dont run unless you really have to check\ntf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names_out())\n\nfinal_df = tf_idf\n\nprint("{} rows".format(final_df.shape[0]))\nfinal_df.T.nlargest(5, 0)\n'

## NN with LSTM approach.... did not use embeddings

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input, RepeatVector, TimeDistributed
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE  # Import SMOTE

In [ ]:
df2=df_train_cleaned.copy()

In [ ]:
# Load Pretrained GloVe Embeddings (100D)
embedding_index = {}
with open("/content/glove.6B.100d.txt", encoding="utf-8") as f:  # Replace with actual GloVe file
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

In [ ]:
# Tokenization & Padding
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df2['crimeaditionalinfo'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df2['crimeaditionalinfo'])
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post')


In [ ]:
# Prepare Embedding Matrix
embedding_dim = 100  # Must match GloVe vector dimension
embedding_matrix = np.zeros((5000, embedding_dim))
for word, i in word_index.items():
    if i < 5000:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
df2['category_encoded'] = label_encoder.fit_transform(df2['category'])


In [ ]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, df2['category_encoded'], test_size=0.2, random_state=42
)

In [ ]:
# Apply SMOTE on Labels
smote = SMOTE(random_state=42, k_neighbors=min(3, len(np.unique(y_train))))
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Convert X_train_resampled back to sequences (Needed if using Tokenizer on text)
X_train_resampled = pad_sequences(X_train_resampled, maxlen=200, padding='post')


In [ ]:
# Build LSTM model
model = Sequential([
    Embedding(5000, 64, input_length=200),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer for classification
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
history=model.fit(X_train_resampled, y_train_resampled, epochs=5, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/5
19908/22783 ━━━━━━━━━━━━━━━━━━━━ 18:57 396ms/step - accuracy: 0.3150 - loss: 1.9808

In [ ]:
# Predict category
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Check for mislabeled data
df_test = df2.iloc[y_test.index].copy() # Use y_test.index to access the correct indices
df_test['predicted_category'] = label_encoder.inverse_transform(y_pred_labels)
df_test['mislabel_flag_lstm'] = df_test['category'] != df_test['predicted_category']


In [ ]:
# Print potential mislabeled cases
mislabeled_cases = df_test[df_test['mislabel_flag_lstm']]
# Count the number of mislabeled cases
mislabeled_count = df_test['mislabel_flag_lstm'].sum()

# Print the count
print(f"Number of mislabeled cases: {mislabeled_count}")

In [ ]:
# Save DataFrame as CSV
df_test.to_csv('relabelled_data_lstm_withsmote.csv', index=False)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [ ]:
# Compute Accuracy, Precision, Recall, and F1-Score
accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels, average='weighted')
recall = recall_score(y_test, y_pred_labels, average='weighted')
f1 = f1_score(y_test, y_pred_labels, average='weighted')

# Print Metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
# Plot Loss
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training & Validation Loss Over Epochs")
plt.legend()
plt.show()